# Coffee Chat Bot (Notebook)

Developed with PyTorch and NLTK.

Based on the following [tutorial](https://www.youtube.com/watch?v=RpWeNzfSUHw) (dataset included).

### 0. Setup

In [1]:
# Imports
import random
import json
import numpy as np

import nltk
#nltk.download('punkt') # Must be called on the first execution
from nltk.stem.porter import PorterStemmer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Util Functions
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return PorterStemmer().stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(word) for word in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)

    for index, word in enumerate(all_words):
        if word in tokenized_sentence:
            bag[index] += 1.0

    return bag

### 1. Load Data

In [3]:
with open("data.json", "r") as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)

    for pattern in intent["patterns"]:
        word = tokenize(pattern)
        all_words.extend(word)
        xy.append((word, tag))

ignore_tokens = ["?", "!", ".", "."]
all_words = [stem(w) for w in all_words if word not in ignore_tokens]

### 2. Data Preparation

In [4]:
# Sort and remove duplicates
all_words = sorted(set(all_words))
tags = sorted(set(tags))

x_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

In [5]:
# Hyper Parameters
batch_size = 8
input_size = len(all_words)
hidden_size = 8
output_size = len(tags)
learning_rate = 0.001
num_epochs = 1000

In [6]:
# Custom Dataset class
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

### 3. Model Definition

In [7]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input):
        out = self.l1(input)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

### 4. Training

In [8]:
model = NN(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        # forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch + 1}/{num_epochs}, loss={loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

Epoch 100/1000, loss=0.8767
Epoch 200/1000, loss=0.4349
Epoch 300/1000, loss=0.0386
Epoch 400/1000, loss=0.0092
Epoch 500/1000, loss=0.0198
Epoch 600/1000, loss=0.0011
Epoch 700/1000, loss=0.0008
Epoch 800/1000, loss=0.0005
Epoch 900/1000, loss=0.0002
Epoch 1000/1000, loss=0.0010
Final loss: 0.0010


### 5. Testing

In [9]:
def chat(sentence):
    # Process sentence
    sentence = tokenize(sentence)
    bag = bag_of_words(sentence, all_words)
    x = bag.reshape(1, bag.shape[0])
    x = torch.from_numpy(x)

    # Let the bot predict the tag
    output = model(x)
    _, pred = torch.max(output, dim=1)
    tag = tags[pred.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][pred]

    # If the tag is recognized with sufficient confidence,
    # then select an adequate answer
    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                return f"Bot: {random.choice(intent['responses'])}"
    else:
        return "Bot: I do not understand..."

In [10]:
user = "Hello there!"
bot = chat(user)

print(f"You: {user}")
print(bot)

You: Hello there!
Bot: Hi there, what can I do for you?


### 6. Persistence

In [13]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

file_name = "chat_bot.pth"
torch.save(data, file_name)

print(f"File saved to {file_name} successfully!")

File saved to chat_bot.pth successfully!
